In [1]:
# CELDA 1: Configuración Inicial y Documentación
"""
Módulo de Prognosis Industrial
=============================

Objetivo: Implementar un sistema automatizado de prognosis para sistemas industriales
que permita detectar y predecir fallas basándose en:
1. Detección de anomalías
2. Análisis de tendencias
3. Evaluación de límites

Estructura del Notebook:
- Fase 1: Preprocesamiento de datos
- Fase 2: Identificación de variables clave
- Fase 3: Aprendizaje de línea base
- Fase 4: Detector de fallas

Autor: [Nombre]
Fecha: [Fecha]
Versión: 1.0
"""

# Importaciones necesarias
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.feature_selection import RFE
from scipy import stats
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Any, Optional
from datetime import datetime
import json
import warnings
from statsmodels.tsa.stattools import acf

# Configuración inicial
plt.style.use('default')
sns.set_theme(style="whitegrid")
warnings.filterwarnings('ignore')

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Configuración de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100

print("Configuración inicial completada exitosamente.")

Configuración inicial completada exitosamente.


In [2]:
# CELDA 2: Fase 1 - Preprocesamiento de Datos
"""
Fase 1: Preprocesamiento de Datos
================================

Esta fase implementa:
1. Carga y validación de datos
2. Control de calidad
3. Normalización y escalado
4. Análisis estadístico inicial

Características principales:
- Manejo de datos faltantes y outliers
- Normalización robusta
- Generación de reportes de calidad
- Preservación de estructura temporal
"""

class DataPreprocessor:
    def __init__(self):
        """Inicializa el preprocesador con configuraciones por defecto"""
        self.scaler = RobustScaler()  # Más robusto que StandardScaler para datos industriales
        self.stats = {}
        self.quality_report = {
            'missing_values': {},
            'infinite_values': {},
            'outliers': {},
            'irrelevant_columns': [],
            'data_types': {},
            'temporal_stats': {},  # Nueva sección para estadísticas temporales
            'warnings': [],
            'errors': []
        }
        self._setup_logging()

    def _setup_logging(self):
        """Configura el sistema de logging"""
        self.logger = logging.getLogger(__name__)

    def process_data(self, data_path: str) -> Tuple[pd.DataFrame, Dict[str, Any]]:
        """
        Procesa los datos desde el archivo fuente.
        
        Args:
            data_path: Ruta al archivo de datos (.csv o .xlsx)
            
        Returns:
            Tuple[pd.DataFrame, Dict]: Datos procesados y reporte de calidad
        """
        try:
            self.logger.info("Iniciando preprocesamiento de datos...")
            
            # 1. Carga y validación inicial
            data = self._load_data(data_path)
            if data is None:
                return None, self.quality_report
            
            # 2. Control de calidad
            data = self._quality_control(data)
            if data is None:
                return None, self.quality_report
                
            # 3. Normalización y escalado
            data_processed = self._normalize_data(data)
            
            # 4. Análisis estadístico
            self._perform_statistical_analysis(data_processed)
            
            # 5. Análisis temporal
            self._analyze_temporal_structure(data_processed)
            
            results = {
                'quality_report': self.quality_report,
                'statistics': self.stats,
                'data_shape': data_processed.shape,
                'columns': list(data_processed.columns),
                'temporal_range': {
                    'start': data_processed.index.min(),
                    'end': data_processed.index.max(),
                    'frequency': self._detect_sampling_frequency(data_processed)
                },
                'processing_summary': {
                    'initial_shape': data.shape,
                    'final_shape': data_processed.shape,
                    'removed_columns': self.quality_report['irrelevant_columns']
                }
            }
            
            self.logger.info("Preprocesamiento completado exitosamente")
            return data_processed, results
            
        except Exception as e:
            self.logger.error(f"Error en preprocesamiento: {str(e)}")
            return None, None

    def _load_data(self, path: str) -> Optional[pd.DataFrame]:
        """Carga los datos desde archivo y configura el índice temporal"""
        try:
            if path.endswith('.xlsx'):
                data = pd.read_excel(path)
            elif path.endswith('.csv'):
                data = pd.read_csv(path)
            else:
                self.logger.error("Formato de archivo no soportado")
                return None
            
            # Verificar y configurar timestamp
            if 'timestamp' in data.columns:
                # Convertir a datetime si no lo es
                if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
                    data['timestamp'] = pd.to_datetime(data['timestamp'])
                
                # Establecer timestamp como índice
                data.set_index('timestamp', inplace=True)
                data.sort_index(inplace=True)  # Ordenar por timestamp
                
                self.logger.info(f"Datos cargados: {data.shape[0]} filas, {data.shape[1]} columnas")
                self.logger.info(f"Rango temporal: {data.index.min()} a {data.index.max()}")
            else:
                self.logger.error("No se encontró columna de timestamp")
                return None
            
            return data
            
        except Exception as e:
            self.logger.error(f"Error en carga de datos: {str(e)}")
            return None

    def _quality_control(self, data: pd.DataFrame) -> Optional[pd.DataFrame]:
        """Realiza control de calidad en los datos preservando estructura temporal"""
        try:
            # 1. Análisis de valores faltantes
            for column in data.columns:
                missing = data[column].isnull().sum()
                self.quality_report['missing_values'][column] = missing
                
                if missing > 0:
                    if missing/len(data) > 0.5:  # Si más del 50% son nulos
                        self.quality_report['irrelevant_columns'].append(column)
                    else:
                        if pd.api.types.is_numeric_dtype(data[column]):
                            # Interpolación temporal para datos numéricos
                            data[column] = data[column].interpolate(method='time')
                        else:
                            data[column] = data[column].fillna(data[column].mode()[0])
            
            # 2. Manejo de infinitos
            numeric_columns = data.select_dtypes(include=[np.number]).columns
            for column in numeric_columns:
                inf_mask = np.isinf(data[column])
                inf_count = inf_mask.sum()
                self.quality_report['infinite_values'][column] = inf_count
                
                if inf_count > 0:
                    data[column] = data[column].replace([np.inf, -np.inf], np.nan)
                    data[column] = data[column].interpolate(method='time')
            
            # 3. Detección de outliers
            for column in numeric_columns:
                z_scores = np.abs(stats.zscore(data[column]))
                outliers = (z_scores > 3).sum()
                self.quality_report['outliers'][column] = outliers
            
            return data
            
        except Exception as e:
            self.logger.error(f"Error en control de calidad: {str(e)}")
            return None

    def _normalize_data(self, data: pd.DataFrame) -> pd.DataFrame:
        """Normaliza las variables numéricas preservando el índice temporal"""
        try:
            numeric_cols = data.select_dtypes(include=[np.number]).columns
            if len(numeric_cols) > 0:
                # Preservar el índice temporal
                scaled_data = self.scaler.fit_transform(data[numeric_cols])
                data[numeric_cols] = scaled_data
            return data
                
        except Exception as e:
            self.logger.error(f"Error en normalización: {str(e)}")
            return data

    def _perform_statistical_analysis(self, data: pd.DataFrame):
        """Realiza análisis estadístico de los datos"""
        try:
            numeric_cols = data.select_dtypes(include=[np.number]).columns
            
            for column in numeric_cols:
                self.stats[column] = {
                    'basic_stats': {
                        'mean': float(data[column].mean()),
                        'std': float(data[column].std()),
                        'min': float(data[column].min()),
                        'max': float(data[column].max()),
                        'median': float(data[column].median())
                    },
                    'distribution': {
                        'skewness': float(data[column].skew()),
                        'kurtosis': float(data[column].kurtosis())
                    }
                }
            
            if len(numeric_cols) > 1:
                self.stats['correlations'] = data[numeric_cols].corr().to_dict()
                
        except Exception as e:
            self.logger.error(f"Error en análisis estadístico: {str(e)}")

    def _analyze_temporal_structure(self, data: pd.DataFrame):
        """Analiza la estructura temporal de los datos"""
        try:
            # Análisis de frecuencia de muestreo
            freq = self._detect_sampling_frequency(data)
            
            self.quality_report['temporal_stats'] = {
                'sampling_frequency': freq,
                'start_time': data.index.min(),
                'end_time': data.index.max(),
                'total_duration': str(data.index.max() - data.index.min()),
                'gaps': self._detect_temporal_gaps(data)
            }
            
        except Exception as e:
            self.logger.error(f"Error en análisis temporal: {str(e)}")

    def _detect_sampling_frequency(self, data: pd.DataFrame) -> str:
        """Detecta la frecuencia de muestreo predominante"""
        try:
            # Calcular diferencias entre timestamps consecutivos
            time_diffs = data.index.to_series().diff()
            most_common_diff = time_diffs.mode()[0]
            
            # Convertir a string formato frecuencia pandas
            seconds = most_common_diff.total_seconds()
            if seconds < 60:
                return f"{int(seconds)}S"
            elif seconds < 3600:
                return f"{int(seconds/60)}T"
            else:
                return f"{int(seconds/3600)}H"
                
        except Exception as e:
            self.logger.error(f"Error detectando frecuencia: {str(e)}")
            return "Unknown"

    def _detect_temporal_gaps(self, data: pd.DataFrame) -> List[Dict[str, Any]]:
        """Detecta gaps significativos en la serie temporal"""
        try:
            time_diffs = data.index.to_series().diff()
            freq = pd.Timedelta(self._detect_sampling_frequency(data))
            gaps = []
            
            # Detectar diferencias mayores a 2 veces la frecuencia normal
            significant_gaps = time_diffs[time_diffs > 2 * freq]
            
            for idx, gap in significant_gaps.items():
                gaps.append({
                    'start': str(idx - gap),
                    'end': str(idx),
                    'duration': str(gap)
                })
                
            return gaps
            
        except Exception as e:
            self.logger.error(f"Error detectando gaps: {str(e)}")
            return []

# Función de prueba
def test_preprocessing():
    """Prueba el preprocesamiento de datos"""
    try:
        print("\n=== PRUEBA DE PREPROCESAMIENTO ===\n")
        
        # Crear instancia del preprocesador
        preprocessor = DataPreprocessor()
        
        # Procesar datos
        data_path = "filtered_consolidated_data_cleaned.xlsx"
        processed_data, results = preprocessor.process_data(data_path)
        
        if processed_data is not None:
            print("\nResumen del procesamiento:")
            print(f"Forma inicial: {results['processing_summary']['initial_shape']}")
            print(f"Forma final: {results['processing_summary']['final_shape']}")
            print(f"\nRango temporal:")
            print(f"Inicio: {results['temporal_range']['start']}")
            print(f"Fin: {results['temporal_range']['end']}")
            print(f"Frecuencia: {results['temporal_range']['frequency']}")
            
            # Mostrar primeras filas de datos procesados
            print("\nMuestra de datos procesados:")
            print(processed_data.head())
            
            return processed_data, results
        else:
            print("Error en el preprocesamiento")
            return None, None
            
    except Exception as e:
        print(f"Error en prueba: {str(e)}")
        return None, None

# Ejecutar prueba
processed_data, preprocessing_results = test_preprocessing()

2024-12-27 09:55:09,464 - INFO - Iniciando preprocesamiento de datos...



=== PRUEBA DE PREPROCESAMIENTO ===



2024-12-27 09:55:15,223 - INFO - Datos cargados: 7141 filas, 56 columnas
2024-12-27 09:55:15,224 - INFO - Rango temporal: 2013-05-04 16:07:00 a 2013-05-09 15:07:00
2024-12-27 09:55:15,581 - INFO - Preprocesamiento completado exitosamente



Resumen del procesamiento:
Forma inicial: (7141, 56)
Forma final: (7141, 56)

Rango temporal:
Inicio: 2013-05-04 16:07:00
Fin: 2013-05-09 15:07:00
Frecuencia: 1T

Muestra de datos procesados:
                     Tensión: L1 (V)  Tensión: L2 (V)  Tensión: L3 (V)  \
timestamp                                                                
2013-05-04 16:07:00        -0.018462        -0.192635        -0.074713   
2013-05-04 16:08:00         0.076923        -0.116147        -0.002874   
2013-05-04 16:09:00         0.092308        -0.099150         0.025862   
2013-05-04 16:10:00         0.089231        -0.113314         0.025862   
2013-05-04 16:11:00         0.113846        -0.099150         0.020115   

                     Tensión: L1 - L2 (V)  Tensión: L2 - L3 (V)  \
timestamp                                                         
2013-05-04 16:07:00             -0.134907             -0.136824   
2013-05-04 16:08:00             -0.053963             -0.064189   
2013-05-04 16:09:00 

In [3]:
# CELDA 3: Fase 2 - Identificación de Variables Clave
"""
Fase 2: Identificación de Variables Clave
=======================================

Objetivos:
1. Identificar variables críticas para prognosis
2. Evaluar importancia multifactorial
3. Analizar patrones temporales
4. Determinar correlaciones significativas
5. Asegurar integración robusta con fases posteriores

Características:
- Análisis multidimensional con preservación temporal
- Categorización adaptativa de variables
- Tracking de cambios en variables críticas
- Validación de estructura de datos
- Integración validada con fases posteriores
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf
from sklearn.preprocessing import RobustScaler
import logging
from typing import Dict, List, Any, Optional, Union
from datetime import datetime, timedelta
from scipy.signal import find_peaks

class VariableAnalyzer:
    def __init__(self):
        self._setup_logging()
        self.scaler = RobustScaler()
        self.results = {
            'importance_scores': {},
            'temporal_patterns': {},
            'correlations': {},
            'selected_variables': {},
            'categories': {
                'critical': {},      
                'monitoring': {},    
                'supporting': {}     
            },
            'metrics': {},
            'data_structure': {      
                'temporal_format': None,
                'sampling_rate': None,
                'variable_metadata': {}
            }
        }
        
        self.state = {
            'last_update': None,
            'variable_history': {},
            'structure_changes': [],
            'validation_status': {}
        }
        
        self.thresholds = {
            'correlation': 0.7,
            'importance': 0.6,
            'stability': 0.5,
            'change_point': 2.0
        }

    def _setup_logging(self):
        self.logger = logging.getLogger(__name__)
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)
            self.logger.setLevel(logging.INFO)

    def analyze_variables(self, data: pd.DataFrame) -> Dict[str, Any]:
        try:
            self.logger.info("Iniciando análisis de variables...")
            
            structure_valid = self._analyze_data_structure(data)
            if not structure_valid:
                raise ValueError("Estructura de datos inválida")
            
            self.data = self._prepare_data(data)
            self.numeric_data = self.data.select_dtypes(include=[np.number])
            
            print(f"Analizando {len(self.numeric_data.columns)} variables numéricas...")
            print(f"Rango temporal: {data.index.min()} a {data.index.max()}")
            
            for column in self.numeric_data.columns:
                self._analyze_single_variable(column)
                self._track_variable_changes(column)
            
            self._analyze_correlations()
            self._update_categories()
            self._validate_results()
            
            self.state['last_update'] = pd.Timestamp.now()
            return self.results
            
        except Exception as e:
            self.logger.error(f"Error en análisis: {str(e)}")
            return None
        
    def _analyze_data_structure(self, data: pd.DataFrame) -> bool:
        try:
            if not isinstance(data.index, pd.DatetimeIndex):
                self.logger.error("Se requiere índice temporal")
                return False
            
            freq = pd.infer_freq(data.index)
            sampling = data.index.to_series().diff().mode()[0]
            
            self.results['data_structure'].update({
                'temporal_format': str(data.index.dtype),
                'sampling_rate': str(sampling),
                'frequency': str(freq),
                'timestamp_format': data.index.strftime('%Y-%m-%d %H:%M:%S')[0]
            })
            
            for column in data.columns:
                self.results['data_structure']['variable_metadata'][column] = {
                    'dtype': str(data[column].dtype),
                    'missing_pct': float(data[column].isnull().mean()),
                    'unique_count': int(data[column].nunique()),
                    'range': {
                        'min': float(data[column].min()) if pd.api.types.is_numeric_dtype(data[column]) else None,
                        'max': float(data[column].max()) if pd.api.types.is_numeric_dtype(data[column]) else None
                    }
                }
            return True
            
        except Exception as e:
            self.logger.error(f"Error en análisis de estructura: {str(e)}")
            return False

    def _prepare_data(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Prepara los datos para el análisis manteniendo la normalización original
        de la fase de preprocesamiento.
        
        Args:
            data: DataFrame con datos ya normalizados de la fase 1
            
        Returns:
            DataFrame: Datos preparados solo con interpolación si es necesaria
        """
        try:
            prepared_data = data.copy()
            
            # Solo realizar interpolación para valores faltantes
            for column in prepared_data.columns:
                if prepared_data[column].isnull().any():
                    prepared_data[column] = prepared_data[column].interpolate(
                        method='time',
                        limit=24,
                        limit_direction='both'
                    )
            
            return prepared_data
                
        except Exception as e:
            self.logger.error(f"Error en preparación de datos: {str(e)}")
            return data

    def _analyze_single_variable(self, column: str):
        try:
            series = self.numeric_data[column]
            importance_score = self._calculate_importance_score(series)
            temporal_patterns = self._analyze_temporal_patterns(series)
            stability = self._calculate_stability(series)
            
            self.results['importance_scores'][column] = {
                'combined_score': importance_score['combined_score'],
                'components': importance_score['components'],
                'temporal_analysis': temporal_patterns,
                'stability': stability
            }
            
        except Exception as e:
            self.logger.error(f"Error en análisis de {column}: {str(e)}")

    def _track_variable_changes(self, column: str):
        """
        Rastrea cambios en las variables a lo largo del tiempo
        
        Args:
            column: Nombre de la variable a rastrear
        """
        try:
            if column not in self.state['variable_history']:
                self.state['variable_history'][column] = []  # Cambiado a lista
            
            current_stats = {
                'timestamp': pd.Timestamp.now(),
                'mean': float(self.numeric_data[column].mean()),
                'std': float(self.numeric_data[column].std()),
                'stability': self._calculate_stability(self.numeric_data[column])
            }
            
            self.state['variable_history'][column].append(current_stats)  # Ahora append funciona
            
        except Exception as e:
            self.logger.error(f"Error en tracking de variable {column}: {str(e)}")

    def _calculate_temporal_stats(self, series: pd.Series) -> Dict[str, float]:
        try:
            # Calcular estadísticas temporales básicas
            acf_values = acf(series.fillna(method='ffill'), nlags=min(len(series)-1, 48))
            
            return {
                'temporal_stability': 1 - series.std() / (series.max() - series.min()),
                'trend_strength': abs(stats.linregress(np.arange(len(series)), series)[2]),
                'autocorrelation': float(acf_values[1]) if len(acf_values) > 1 else 0.0
            }
        except Exception as e:
            self.logger.error(f"Error en estadísticas temporales: {str(e)}")
            return {'temporal_stability': 0.0, 'trend_strength': 0.0, 'autocorrelation': 0.0}

    def _calculate_importance_score(self, series: pd.Series) -> Dict[str, Any]:
        try:
            variance = float(series.var())
            std = float(series.std())
            range_size = float(series.max() - series.min())
            
            temporal_stats = self._calculate_temporal_stats(series)
            
            weights = {
                'variance': 0.15,
                'temporal_stability': 0.25,
                'trend_strength': 0.3,
                'autocorrelation': 0.3
            }
            
            components = {
                'variance': min(variance / range_size, 1.0),
                'temporal_stability': temporal_stats['temporal_stability'],
                'trend_strength': temporal_stats['trend_strength'],
                'autocorrelation': abs(temporal_stats['autocorrelation'])
            }
            
            combined_score = sum(components[k] * weights[k] for k in weights)
            
            return {
                'combined_score': float(min(max(combined_score, 0), 1)),
                'components': components
            }
            
        except Exception as e:
            self.logger.error(f"Error en cálculo de importancia: {str(e)}")
            return {'combined_score': 0.0, 'components': {}}

    def _estimate_seasonality_period(self, series: pd.Series) -> int:
        try:
            n_lags = min(len(series)-1, 168)  # Máximo 1 semana
            acf_values = acf(series.fillna(method='ffill'), nlags=n_lags)
            peaks, _ = find_peaks(acf_values, height=0.3)
            return int(peaks[0]) if len(peaks) > 0 else 24
        except Exception as e:
            self.logger.error(f"Error en estimación de estacionalidad: {str(e)}")
            return 24

    def _analyze_temporal_patterns(self, series: pd.Series) -> Dict[str, Any]:
        try:
            period = self._estimate_seasonality_period(series)
            decomposition = seasonal_decompose(
                series.fillna(method='ffill'),
                period=period
            )
            
            trend = decomposition.trend
            seasonal = decomposition.seasonal
            residual = decomposition.resid
            
            change_points = self._detect_change_points(series)
            
            return {
                'trend_strength': float(1 - (residual.std() / series.std())),
                'seasonal_strength': float(1 - (residual.std() / seasonal.std())),
                'seasonality_period': int(period),
                'change_points': change_points
            }
            
        except Exception as e:
            self.logger.error(f"Error en análisis temporal: {str(e)}")
            return {}

    def _detect_change_points(self, series: pd.Series) -> List[Dict[str, Any]]:
        try:
            rolling_mean = series.rolling(window=24).mean()
            rolling_std = series.rolling(window=24).std()
            z_scores = abs(series - rolling_mean) / rolling_std
            
            change_points = []
            for idx in range(len(z_scores)):
                if z_scores[idx] > self.thresholds['change_point']:
                    change_points.append({
                        'timestamp': str(series.index[idx]),
                        'score': float(z_scores[idx])
                    })
            
            return change_points
        except Exception as e:
            self.logger.error(f"Error en detección de cambios: {str(e)}")
            return []

   

    def _analyze_correlations(self):
        try:
            corr_matrix = self.numeric_data.corr()
            self.results['correlations'] = {}
            
            for col1 in corr_matrix.columns:
                self.results['correlations'][col1] = {}
                for col2 in corr_matrix.columns:
                    if col1 != col2 and abs(corr_matrix.loc[col1, col2]) > self.thresholds['correlation']:
                        self.results['correlations'][col1][col2] = float(corr_matrix.loc[col1, col2])
                        
        except Exception as e:
            self.logger.error(f"Error en análisis de correlaciones: {str(e)}")

    def _get_current_category(self, var: str) -> str:
        for category, vars_dict in self.results['categories'].items():
            if var in vars_dict:
                return category
        return 'supporting'

    def _track_category_change(self, var: str, old_cat: str, new_cat: str):
        if var not in self.state['variable_history']:
            self.state['variable_history'][var] = []
            
        self.state['variable_history'][var].append({
            'timestamp': pd.Timestamp.now(),
            'old_category': old_cat,
            'new_category': new_cat
        })

    def _calculate_stability(self, series: pd.Series) -> float:
        """
        Calcula la estabilidad temporal de una serie
        
        Args:
            series: Serie temporal a analizar
            
        Returns:
            float: Score de estabilidad entre 0 y 1
        """
        try:
            rolling_std = series.rolling(window=24).std()
            stability = 1 - (rolling_std.mean() / series.std())
            return float(max(0, min(1, stability)))
        except Exception as e:
            self.logger.error(f"Error en cálculo de estabilidad: {str(e)}")
            return 0.0

    def _update_categories(self):
        try:
            importance_scores = {
                k: v['combined_score'] 
                for k, v in self.results['importance_scores'].items()
            }
            sorted_vars = sorted(importance_scores.items(), 
                               key=lambda x: x[1], 
                               reverse=True)
            
            scores = np.array([score for _, score in sorted_vars])
            critical_threshold = np.percentile(scores, 80)
            monitoring_threshold = np.percentile(scores, 50)
            
            for var, score in sorted_vars:
                old_category = self._get_current_category(var)
                
                if score >= critical_threshold:
                    new_category = 'critical'
                elif score >= monitoring_threshold:
                    new_category = 'monitoring'
                else:
                    new_category = 'supporting'
                
                if old_category != new_category:
                    self._track_category_change(var, old_category, new_category)
                
                self.results['categories'][new_category][var] = {
                    'importance_score': score,
                    'temporal_stability': self._calculate_stability(self.numeric_data[var]),
                    'last_update': pd.Timestamp.now()
                }
            
            self.results['selected_variables'] = {
                var: score for var, score in sorted_vars[:int(len(sorted_vars) * 0.2)]
            }
            
        except Exception as e:
            self.logger.error(f"Error en actualización de categorías: {str(e)}")

    def _validate_results(self):
        try:
            validation_status = {
                'complete': True,
                'warnings': []
            }
            
            # Validar variables críticas
            if not self.results['categories']['critical']:
                validation_status['warnings'].append("No se identificaron variables críticas")
                validation_status['complete'] = False
            
            # Validar análisis temporal
            for var in self.results['selected_variables']:
                if var not in self.results['temporal_patterns']:
                    validation_status['warnings'].append(f"Falta análisis temporal para {var}")
                    validation_status['complete'] = False
            
            self.state['validation_status']['results'] = validation_status
            
        except Exception as e:
            self.logger.error(f"Error en validación de resultados: {str(e)}")

    def validate_integration(self, phase: str) -> Dict[str, Any]:
        try:
            validation = {
                'status': True,
                'warnings': [],
                'recommendations': []
            }
            
            if phase == 'baseline':
                self._validate_baseline_requirements(validation)
            elif phase == 'prediction':
                self._validate_prediction_requirements(validation)
            
            self.state['validation_status'][phase] = validation
            return validation
            
        except Exception as e:
            self.logger.error(f"Error en validación: {str(e)}")
            return {'status': False, 'error': str(e)}

    def _validate_baseline_requirements(self, validation: Dict):
        if not self.results['selected_variables']:
            validation['status'] = False
            validation['warnings'].append("No hay variables seleccionadas")
        
        for var in self.results['selected_variables']:
            if var not in self.results['temporal_patterns']:
                validation['warnings'].append(f"Falta análisis temporal para {var}")

    def _validate_prediction_requirements(self, validation: Dict):
        if not self.results['categories']['critical']:
            validation['status'] = False
            validation['warnings'].append("No hay variables críticas identificadas")

def test_variable_analysis(data: pd.DataFrame):
    try:
        print("\n=== PRUEBA DE ANÁLISIS DE VARIABLES ===\n")
        
        analyzer = VariableAnalyzer()
        results = analyzer.analyze_variables(data)
        
        if results:
            print("\nVariables más importantes:")
            for var, score in sorted(
                results['selected_variables'].items(), 
                key=lambda x: x[1], 
                reverse=True
            ):
                print(f"{var}: {score:.3f}")
            
            baseline_validation = analyzer.validate_integration('baseline')
            prediction_validation = analyzer.validate_integration('prediction')
            
            print("\nEstado de validación:")
            print(f"Baseline: {'✓' if baseline_validation['status'] else '✗'}")
            print(f"Prediction: {'✓' if prediction_validation['status'] else '✗'}")
            
            return analyzer
        else:
            print("Error en análisis de variables")
            return None
            
    except Exception as e:
        print(f"Error en prueba: {str(e)}")
        return None

if 'processed_data' in globals():
    variable_analyzer = test_variable_analysis(processed_data)
else:
    print("Error: Ejecutar primero el preprocesamiento de datos")

2024-12-27 09:55:15,867 - __main__ - INFO - Iniciando análisis de variables...
2024-12-27 09:55:15,867 - INFO - Iniciando análisis de variables...



=== PRUEBA DE ANÁLISIS DE VARIABLES ===

Analizando 56 variables numéricas...
Rango temporal: 2013-05-04 16:07:00 a 2013-05-09 15:07:00

Variables más importantes:
Armónicos IL1: Armónico 3 (%IL1): 0.730
Armónicos IL1: Armónico 2 (%IL1): 0.706
Distorsión armónica: IL1 (%I THD): 0.699
Armónicos VL3: Armónico 5 (%VL3): 0.685
Armónicos VL2: Armónico 5 (%VL2): 0.679
Armónicos IL2: Armónico 7 (%IL2): 0.670
Armónicos VL1: Armónico 5 (%VL1): 0.670
Armónicos IL1: Armónico 7 (%IL1): 0.658
Corriente: L1 (A): 0.654
Armónicos IL1: Armónico 5 (%IL1): 0.639
Tensión: L2 - L3 (V): 0.637

Estado de validación:
Baseline: ✓
Prediction: ✓


In [4]:
# CELDA 4: Fase 3 - Aprendizaje de Línea Base
"""
Fase 3: Aprendizaje de Línea Base
================================

Objetivos:
1. Establecer patrones normales de comportamiento para todas las variables clave
2. Crear modelos de referencia adaptativos
3. Definir límites dinámicos de operación
4. Implementar detección de desviaciones
5. Validar robustez del modelo base
6. Preparar base para el sistema de prognosis

Características principales:
- Procesamiento paralelo de todas las variables clave
- Límites dinámicos adaptativos
- Validación robusta de modelos
- Métricas agregadas para evaluación global
- Preparación para detección temprana de fallas
"""

# Importaciones necesarias
import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from scipy import stats
import statsmodels.api as sm
from typing import Dict, List, Tuple, Any, Optional, Callable
import logging
from datetime import datetime, timedelta
import json
import warnings
from scipy.stats import norm
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

class BaselineLearner:
    """
    Clase para el aprendizaje de línea base del sistema.
    
    Establece patrones normales de comportamiento y límites adaptativos
    para todas las variables clave del sistema, preparando la base para
    la detección temprana de fallas.
    """

    def __init__(self, n_periods: int = 24):
        """
        Inicializa el aprendiz de línea base.
        
        Args:
            n_periods: Número de períodos para estacionalidad (default=24 horas)
        """
        self._setup_logging()
        self.n_periods = n_periods
        self.models = {}
        self.limits = {}
        self.stats = {}
        self.validation = {}
        
        # Parámetros optimizados para el sistema
        self.params = {
            'confidence_level': 0.80,    # Nivel de confianza para límites
            'min_samples': 50,           # Mínimo de muestras para modelado
            'contamination': 0.1,        # Factor de contaminación para detección de outliers
            'window_size': 24,           # Tamaño de ventana para límites dinámicos
            'n_splits': 3,               # Número de splits para validación
            'coverage_threshold': 0.5,    # Umbral mínimo de cobertura aceptable
            'max_workers': 4,            # Máximo de workers para procesamiento paralelo
            'maxiter': 300,              # Máximo de iteraciones para ajuste de modelo
            'model_order': (1, 0, 1),    # Orden del modelo SARIMA
            'seasonal_order': (0, 1, 1)  # Orden estacional del modelo
        }

    def _setup_logging(self):
        """Configura el sistema de logging."""
        self.logger = logging.getLogger(__name__)
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter(
                '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
            )
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)
            self.logger.setLevel(logging.INFO)

    def learn_baseline(self, data: pd.DataFrame, selected_vars: Dict[str, float]) -> Dict[str, Any]:
        """
        Aprende la línea base para todas las variables seleccionadas.
        
        Args:
            data: DataFrame con los datos históricos
            selected_vars: Diccionario de variables seleccionadas y sus scores
            
        Returns:
            Dict con resultados del aprendizaje
        """
        try:
            self.logger.info(f"Iniciando aprendizaje de línea base para {len(selected_vars)} variables...")
            results = {
                'models': {},
                'limits': {},
                'metrics': {},
                'validation': {},
                'summary': {}
            }
            
            # Procesamiento paralelo de todas las variables
            with ThreadPoolExecutor(max_workers=self.params['max_workers']) as executor:
                future_to_var = {
                    executor.submit(self._learn_variable_baseline, data[var], var): var 
                    for var in selected_vars
                }
                
                for future in concurrent.futures.as_completed(future_to_var):
                    var = future_to_var[future]
                    try:
                        model_results = future.result()
                        if model_results:
                            results['models'][var] = model_results['model']
                            results['limits'][var] = model_results['limits']
                            results['metrics'][var] = model_results['metrics']
                            results['validation'][var] = model_results['validation']
                            self.logger.info(f"Variable {var} procesada exitosamente")
                    except Exception as e:
                        self.logger.error(f"Error procesando variable {var}: {str(e)}")

            # Agregar métricas globales y validación
            results['summary'] = self._aggregate_metrics(results)
            self._validate_results(results)
            
            return results
            
        except Exception as e:
            self.logger.error(f"Error en aprendizaje de línea base: {str(e)}")
            return None
        
    def _learn_variable_baseline(self, series: pd.Series, var_name: str) -> Dict[str, Any]:
        """
        Aprende la línea base para una variable específica.
        
        Args:
            series: Serie temporal de la variable
            var_name: Nombre de la variable
            
        Returns:
            Dict con resultados del modelo
        """
        try:
            self.logger.debug(f"Iniciando procesamiento de {var_name}")
            
            # Limpieza y preparación
            clean_series = self._remove_outliers(series)
            stats = self._calculate_base_stats(clean_series)
            
            # Ajuste del modelo
            model = self._fit_baseline_model(clean_series)
            
            if model is not None:
                # Cálculo de límites y validación
                limits = self._calculate_dynamic_limits(clean_series, stats)
                residuals = self._analyze_residuals(model, clean_series)
                validation = self._validate_variable_model(model, clean_series)
                
                return {
                    'model': model,
                    'limits': limits,
                    'metrics': stats,
                    'residuals': residuals,
                    'validation': validation
                }
            return None
            
        except Exception as e:
            self.logger.error(f"Error en aprendizaje de variable {var_name}: {str(e)}")
            return None

    def _remove_outliers(self, series: pd.Series) -> pd.Series:
        """
        Elimina outliers usando IsolationForest.
        
        Args:
            series: Serie temporal a limpiar
            
        Returns:
            Serie temporal sin outliers
        """
        try:
            iso = IsolationForest(
                contamination=self.params['contamination'],
                random_state=42,
                n_jobs=-1  # Usar todos los cores disponibles
            )
            
            X = series.values.reshape(-1, 1)
            yhat = iso.fit_predict(X)
            mask = yhat != -1
            
            cleaned_series = series[mask]
            self.logger.debug(f"Outliers removidos: {len(series) - len(cleaned_series)}")
            
            return cleaned_series
            
        except Exception as e:
            self.logger.error(f"Error en remoción de outliers: {str(e)}")
            return series

    def _analyze_residuals(self, model: Any, series: pd.Series) -> Dict[str, float]:
        """
        Analiza los residuos del modelo para validar ajuste
        """
        try:
            # Obtener residuos del modelo de forma segura
            if hasattr(model, 'resid'):
                residuals = model.resid
            else:
                # Calcular residuos manualmente si no están disponibles
                predictions = model.get_prediction(start=0).predicted_mean
                residuals = series - predictions
                
            # Asegurarse de que los residuos son finitos
            residuals = residuals[np.isfinite(residuals)]
            
            if len(residuals) == 0:
                raise ValueError("No hay residuos válidos para analizar")
                
            residual_metrics = {
                'mean': float(np.mean(residuals)),
                'std': float(np.std(residuals)),
                'skew': float(stats.skew(residuals)),
                'kurtosis': float(stats.kurtosis(residuals))
            }
            
            # Calcular tests estadísticos de forma segura
            try:
                residual_metrics['jarque_bera'] = float(stats.jarque_bera(residuals)[0])
            except:
                residual_metrics['jarque_bera'] = np.inf
                
            try:
                residual_metrics['ljung_box'] = float(
                    sm.stats.diagnostic.acorr_ljungbox(residuals, lags=[10], return_df=False)[1][0]
                )
            except:
                residual_metrics['ljung_box'] = 0.0
                
            # Actualizar métricas derivadas
            residual_metrics.update({
                'normality': residual_metrics['jarque_bera'] < 5.99,
                'autocorr': residual_metrics['ljung_box'] > 0.05,
                'stationarity': float(sm.tsa.stattools.adfuller(residuals)[1])
            })
            
            return residual_metrics
            
        except Exception as e:
            self.logger.error(f"Error en análisis de residuos: {str(e)}")
            # Retornar valores por defecto en caso de error
            return {
                'mean': 0.0,
                'std': np.inf,
                'skew': 0.0,
                'kurtosis': 0.0,
                'jarque_bera': np.inf,
                'ljung_box': 0.0,
                'normality': False,
                'autocorr': False,
                'stationarity': 1.0
            }
        
    def _validate_variable_model(self, model: Any, series: pd.Series) -> Dict[str, Any]:
        """
        Valida el modelo ajustado para una variable específica.
        """
        try:
            # Debug inicial
            self.logger.info(f"""
            === Iniciando validación ===
            Serie original:
            - Shape: {series.shape}
            - Valores finitos: {np.sum(np.isfinite(series))}
            - Rango: [{series.min():.3f}, {series.max():.3f}]
            """)
            
            # Asegurar que la serie tiene índice temporal
            if not isinstance(series.index, pd.DatetimeIndex):
                self.logger.warning("Serie sin índice temporal - usando RangeIndex")
                series.index = pd.date_range(
                    start='2000-01-01', 
                    periods=len(series), 
                    freq='H'
                )
            
            # Remover NaN/inf y verificar
            valid_series = series[np.isfinite(series)].copy()
            self.logger.info(f"""
            Serie limpia:
            - Shape: {valid_series.shape}
            - Rango índices: [{valid_series.index.min()}, {valid_series.index.max()}]
            """)
            
            if len(valid_series) == 0:
                raise ValueError("No hay puntos válidos en la serie")
            
            # Obtener predicciones
            try:
                # Ajustar nuevo modelo solo para el período de validación
                validation_model = sm.tsa.SARIMAX(
                    valid_series,
                    order=self.params['model_order'],
                    seasonal_order=self.params['seasonal_order'] + (self.n_periods,),
                    enforce_stationarity=False,
                    enforce_invertibility=False
                ).fit(disp=False)
                
                # Generar predicciones in-sample
                predictions = validation_model.get_prediction(
                    start=valid_series.index[0],
                    end=valid_series.index[-1]
                )
                
                pred_mean = predictions.predicted_mean
                conf_int = predictions.conf_int(alpha=1-self.params['confidence_level'])
                
                self.logger.info(f"""
                Predicciones generadas:
                - Shape predicción: {pred_mean.shape}
                - Shape intervalos: {conf_int.shape}
                - Valores finitos predicción: {np.sum(np.isfinite(pred_mean))}
                """)
                
            except Exception as e:
                self.logger.error(f"Error en predicciones: {str(e)}")
                raise
            
            # Alinear índices
            valid_mask = (
                pred_mean.index.isin(valid_series.index) &
                conf_int.index.isin(valid_series.index)
            )
            
            if np.sum(valid_mask) > 0:
                # Calcular métricas
                aligned_actual = valid_series[pred_mean.index[valid_mask]]
                aligned_pred = pred_mean[valid_mask]
                aligned_lower = conf_int.iloc[valid_mask, 0]
                aligned_upper = conf_int.iloc[valid_mask, 1]
                
                mse = np.mean((aligned_actual - aligned_pred) ** 2)
                mae = np.mean(np.abs(aligned_actual - aligned_pred))
                
                # Calcular cobertura
                in_bounds = (
                    (aligned_actual >= aligned_lower) & 
                    (aligned_actual <= aligned_upper)
                )
                coverage = float(np.mean(in_bounds))
                
                self.logger.info(f"""
                Métricas finales:
                - MSE: {mse:.6f}
                - MAE: {mae:.6f}
                - Cobertura: {coverage:.3f}
                - Puntos dentro de límites: {np.sum(in_bounds)}
                """)
                
                return {
                    'mse': float(mse),
                    'mae': float(mae),
                    'coverage': float(coverage),
                    'n_samples': int(np.sum(valid_mask)),
                    'n_coverage_samples': int(np.sum(in_bounds))
                }
            
            self.logger.warning("No hay puntos válidos alineados para calcular métricas")
            return {
                'mse': np.inf,
                'mae': np.inf,
                'coverage': 0.0,
                'n_samples': 0,
                'n_coverage_samples': 0
            }
                
        except Exception as e:
            self.logger.error(f"Error en validación: {str(e)}")
            return {
                'mse': np.inf,
                'mae': np.inf,
                'coverage': 0.0,
                'n_samples': 0,
                'n_coverage_samples': 0
            }
       
    def _calculate_base_stats(self, series: pd.Series) -> Dict[str, float]:
        """Calcula estadísticas base de la serie temporal."""
        try:
            stats = {
                'mean': float(series.mean()),
                'std': float(series.std()),
                'median': float(series.median()),
                'q1': float(series.quantile(0.25)),
                'q3': float(series.quantile(0.75)),
                'min': float(series.min()),
                'max': float(series.max())
            }
            
            stats.update({
                'iqr': stats['q3'] - stats['q1'],
                'range': stats['max'] - stats['min'],
                'cv': stats['std'] / stats['mean'] if stats['mean'] != 0 else np.inf
            })
            
            return stats
        except Exception as e:
            self.logger.error(f"Error en cálculo de estadísticas: {str(e)}")
            return {}
        
    def _fit_baseline_model(self, series: pd.Series) -> Optional[Any]:
        """Ajusta modelo SARIMA a la serie temporal."""
        try:
            if len(series) < self.params['min_samples']:
                self.logger.warning(f"Serie muy corta: {len(series)} < {self.params['min_samples']}")
                return None
            
            model = sm.tsa.SARIMAX(
                series,
                order=self.params['model_order'],
                seasonal_order=self.params['seasonal_order'] + (self.n_periods,),
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            
            fitted_model = model.fit(
                disp=False,
                maxiter=self.params['maxiter'],
                method='lbfgs',
                low_memory=True
            )
            
            return fitted_model
        except Exception as e:
            self.logger.error(f"Error en ajuste de modelo: {str(e)}")
            return None

    def _calculate_dynamic_limits(self, series: pd.Series, base_stats: Dict[str, float]) -> Dict[str, Any]:
        """Calcula límites dinámicos y estáticos."""
        try:
            z_score = norm.ppf(self.params['confidence_level'])
            
            # Límites estáticos basados en estadísticas globales
            static_limits = {
                'upper': base_stats['mean'] + z_score * base_stats['std'],
                'lower': base_stats['mean'] - z_score * base_stats['std']
            }
            
            # Límites dinámicos basados en ventana móvil
            rolling_mean = series.rolling(window=self.params['window_size']).mean()
            rolling_std = series.rolling(window=self.params['window_size']).std()
            
            dynamic_upper = rolling_mean + z_score * rolling_std
            dynamic_lower = rolling_mean - z_score * rolling_std
            
            return {
                'static': static_limits,
                'dynamic': {
                    'upper': dynamic_upper.to_dict(),
                    'lower': dynamic_lower.to_dict()
                }
            }
        except Exception as e:
            self.logger.error(f"Error en cálculo de límites: {str(e)}")
            return {}

    def _summarize_coverage(self, results: Dict[str, Any]) -> Dict[str, Any]:
        """Genera resumen de cobertura para todas las variables."""
        try:
            # Verificar que tenemos resultados válidos
            if not results or 'validation' not in results:
                return {
                    'mean_coverage': 0.0,
                    'variables_below_threshold': [],
                    'coverage_threshold': self.params['coverage_threshold']
                }
            
            # Obtener coberturas de cada variable
            coverages = {}
            for var, val in results['validation'].items():
                if isinstance(val, dict) and 'coverage' in val:
                    coverages[var] = val['coverage']
            
            # Identificar variables bajo el umbral
            below_threshold = [
                var for var, coverage in coverages.items() 
                if coverage < self.params['coverage_threshold']
            ]
            
            # Calcular media de cobertura
            mean_coverage = np.mean(list(coverages.values())) if coverages else 0.0
            
            return {
                'mean_coverage': float(mean_coverage),
                'variables_below_threshold': below_threshold,
                'coverage_threshold': self.params['coverage_threshold']
            }
            
        except Exception as e:
            self.logger.error(f"Error en resumen de cobertura: {str(e)}")
            return {
                'mean_coverage': 0.0,
                'variables_below_threshold': [],
                'coverage_threshold': self.params['coverage_threshold']
            }

    def _aggregate_metrics(self, results: Dict[str, Any]) -> Dict[str, Any]:
        """Agrega métricas de todos los modelos."""
        try:
            # Asegurarse de que hay resultados válidos
            if not results or not results.get('models'):
                raise ValueError("No hay resultados para agregar")
                
            metrics = {
                'total_variables': len(results['models']),
                'valid_models': sum(1 for m in results['models'].values() if m is not None),
            }
            
            # Calcular métricas promedio de forma segura
            validation_values = [v for v in results.get('validation', {}).values() 
                            if isinstance(v, dict)]
            
            if validation_values:
                metrics['average_metrics'] = {
                    'mse': np.mean([v.get('mse', np.inf) 
                                for v in validation_values 
                                if np.isfinite(v.get('mse', np.inf))]),
                    'mae': np.mean([v.get('mae', np.inf) 
                                for v in validation_values 
                                if np.isfinite(v.get('mae', np.inf))]),
                    'coverage': np.mean([v.get('coverage', 0.0) 
                                    for v in validation_values 
                                    if np.isfinite(v.get('coverage', 0.0))])
                }
            else:
                metrics['average_metrics'] = {
                    'mse': np.inf,
                    'mae': np.inf,
                    'coverage': 0.0
                }
                
            # Agregar resumen de cobertura
            metrics['coverage_summary'] = self._summarize_coverage(results)
            
            return metrics
            
        except Exception as e:
            self.logger.error(f"Error en agregación de métricas: {str(e)}")
            return {
                'total_variables': 0,
                'valid_models': 0,
                'average_metrics': {'mse': np.inf, 'mae': np.inf, 'coverage': 0.0},
                'coverage_summary': {'mean_coverage': 0.0, 'variables_below_threshold': []}
            }
    def _validate_results(self, results: Dict[str, Any]):
        """Valida resultados globales y genera reporte."""
        try:
            # Verificar que tenemos resultados válidos
            if not results or 'summary' not in results:
                results['validation'] = {
                    'global': {
                        'status': False,
                        'warnings': ['No hay resultados válidos para validar'],
                        'metrics': {},
                        'coverage_summary': {}
                    }
                }
                return
                
            validation = {
                'status': True,
                'warnings': [],
                'metrics': results.get('summary', {}),
                'coverage_summary': results.get('summary', {}).get('coverage_summary', {})
            }
            
            # Validaciones específicas
            total_vars = results['summary'].get('total_variables', 0)
            valid_models = results['summary'].get('valid_models', 0)
            
            if valid_models < total_vars:
                validation['status'] = False
                validation['warnings'].append(
                    f"Solo {valid_models} de {total_vars} modelos son válidos"
                )
            
            coverage_summary = results['summary'].get('coverage_summary', {})
            below_threshold = coverage_summary.get('variables_below_threshold', [])
            
            if below_threshold:
                validation['warnings'].append(
                    f"Variables con baja cobertura: {', '.join(below_threshold)}"
                )
            
            results['validation']['global'] = validation
            
        except Exception as e:
            self.logger.error(f"Error en validación de resultados: {str(e)}")
            results['validation'] = {
                'global': {
                    'status': False,
                    'warnings': [f'Error en validación: {str(e)}'],
                    'metrics': {},
                    'coverage_summary': {}
                }
            }

def test_baseline_learning(data: pd.DataFrame, selected_vars: Dict[str, float]):
    """Función de prueba para el aprendizaje de línea base."""
    try:
        print("\n=== PRUEBA DE APRENDIZAJE DE LÍNEA BASE ===\n")
        
        learner = BaselineLearner()
        results = learner.learn_baseline(data, selected_vars)
        
        if results:
            print("\nResumen Global:")
            print(f"Total variables procesadas: {results['summary']['total_variables']}")
            print(f"Modelos válidos: {results['summary']['valid_models']}")
            
            print(f"\nMétricas promedio:")
            for metric, value in results['summary']['average_metrics'].items():
                print(f"  {metric}: {value:.3f}")
            
            print("\nCobertura:")
            coverage = results['summary']['coverage_summary']
            print(f"  Media: {coverage['mean_coverage']:.3f}")
            if coverage['variables_below_threshold']:
                print("\nVariables con baja cobertura:")
                for var in coverage['variables_below_threshold']:
                    print(f"  - {var}")
            
            print("\nValidación global:")
            global_validation = results['validation']['global']
            print(f"Estado: {'✓' if global_validation['status'] else '✗'}")
            
            if global_validation['warnings']:
                print("\nAdvertencias:")
                for warning in global_validation['warnings']:
                    print(f"- {warning}")
            
            return learner
        else:
            print("Error en aprendizaje de línea base")
            return None
            
    except Exception as e:
        print(f"Error en prueba: {str(e)}")
        return None

# Ejecución condicional
if 'processed_data' in globals() and 'variable_analyzer' in globals():
    if variable_analyzer and variable_analyzer.results['selected_variables']:
        recent_data = processed_data.iloc[-1000:]  # Últimos 1000 registros
        baseline_learner = test_baseline_learning(
            recent_data,
            variable_analyzer.results['selected_variables']
        )
    else:
        print("Error: No hay variables seleccionadas")
else:
    print("Error: Ejecutar primero el análisis de variables")

2024-12-27 09:55:24,585 - __main__ - INFO - Iniciando aprendizaje de línea base para 11 variables...
2024-12-27 09:55:24,585 - INFO - Iniciando aprendizaje de línea base para 11 variables...



=== PRUEBA DE APRENDIZAJE DE LÍNEA BASE ===



2024-12-27 09:56:20,603 - __main__ - INFO - 
            === Iniciando validación ===
            Serie original:
            - Shape: (914,)
            - Valores finitos: 914
            - Rango: [-0.070, 0.296]
            
2024-12-27 09:56:20,603 - INFO - 
            === Iniciando validación ===
            Serie original:
            - Shape: (914,)
            - Valores finitos: 914
            - Rango: [-0.070, 0.296]
            
2024-12-27 09:56:20,873 - __main__ - INFO - 
            Serie limpia:
            - Shape: (914,)
            - Rango índices: [2013-05-08 22:28:00, 2013-05-09 15:06:00]
            
2024-12-27 09:56:20,873 - INFO - 
            Serie limpia:
            - Shape: (914,)
            - Rango índices: [2013-05-08 22:28:00, 2013-05-09 15:06:00]
            
2024-12-27 09:56:21,823 - __main__ - INFO - 
            === Iniciando validación ===
            Serie original:
            - Shape: (933,)
            - Valores finitos: 933
            - Rango: [-


Resumen Global:
Total variables procesadas: 11
Modelos válidos: 11

Métricas promedio:
  mse: 0.004
  mae: 0.037
  coverage: 0.868

Cobertura:
  Media: 0.868

Validación global:
Estado: ✓


In [5]:
# CELDA 5: Fase 4 - Sistema de Prognosis Industrial
"""
Fase 4: Sistema de Prognosis Industrial
=====================================

Sistema de prognosis industrial con:
1. Detección de anomalías y análisis de tendencias
2. Evaluación frente a límites adaptativos
3. Predicción y alertas tempranas
4. Métricas de rendimiento y KPIs
5. Visualización y reportes
"""

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor, IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, precision_score, recall_score
import logging
from typing import Dict, List, Tuple, Any, Optional
from datetime import datetime
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from dataclasses import dataclass
import json
import warnings
warnings.filterwarnings('ignore')

@dataclass
class PrognosisConfig:
    """Configuración del sistema de prognosis."""
    # Parámetros de entrenamiento
    n_splits: int = 5
    test_size: float = 0.2
    confidence_level: float = 0.95
    
    # Ventanas temporales
    window_size: int = 24
    forecast_horizon: int = 12
    lags: List[int] = None
    
    # Umbrales ajustados
    alert_threshold: float = 0.8        # Aumentado para reducir falsos positivos
    anomaly_contamination: float = 0.05 # Reducido para ser más selectivo
    trend_threshold: float = 0.1        # Aumentado para detectar tendencias más significativas
    
    # Métricas objetivo ajustadas
    target_precision: float = 0.80      # Más realista
    target_recall: float = 0.75         # Más realista
    max_false_positives: float = 0.1    # Aumentado ligeramente
    
    def __post_init__(self):
        if self.lags is None:
            self.lags = [1, 2, 3, 6, 12, 24]

class PrognosisSystem:
    """Sistema de prognosis industrial."""
    
    def __init__(self, config: Optional[PrognosisConfig] = None):
        """Inicializa el sistema de prognosis."""
        self.config = config or PrognosisConfig()
        self._setup_logging()
        self._initialize_storage()
        
    def _setup_logging(self):
        """Configura sistema de logging."""
        self.logger = logging.getLogger(__name__)
        if not self.logger.handlers:
            handler = logging.StreamHandler()
            formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
            handler.setFormatter(formatter)
            self.logger.addHandler(handler)
            self.logger.setLevel(logging.INFO)

    def _initialize_storage(self):
        """Inicializa almacenamiento de modelos y métricas."""
        self.models = {}
        self.anomaly_detectors = {}
        self.scalers = {}
        self.trend_detectors = {}
        self.metrics = {}
        self.limits = {}
        self.baselines = {}
        self.predictions_history = {}
        self.alerts_history = []

    def train_models(self, data: pd.DataFrame, variables: List[str]) -> Dict[str, Any]:
        """Entrena modelos para las variables seleccionadas."""
        try:
            self.logger.info(f"Iniciando entrenamiento para {len(variables)} variables")
            training_results = {}
            
            for variable in variables:
                results = self._train_single_model(data, variable)
                training_results[variable] = results
                
            self._validate_training(training_results)
            self.logger.info("Entrenamiento completado exitosamente")
            return training_results
            
        except Exception as e:
            self.logger.error(f"Error en entrenamiento: {str(e)}")
            raise

    def _train_single_model(self, data: pd.DataFrame, variable: str) -> Dict[str, Any]:
        """Entrena modelo para una variable específica."""
        try:
            # Preparar datos
            X, y = self._prepare_features(data, variable)
            
            # Entrenar detector de anomalías
            anomaly_detector = self._train_anomaly_detector(X)
            normal_mask = anomaly_detector.predict(X) == 1
            X_clean = X[normal_mask]
            y_clean = y[normal_mask]
            
            # Escalar features
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_clean)
            
            # Entrenar modelo predictivo
            model = self._train_predictor(X_scaled, y_clean)
            
            # Calcular línea base
            baseline = self._calculate_baseline(data[variable])
            
            # Validar y guardar
            metrics = self._validate_model(model, X_scaled, y_clean)
            
            # Almacenar componentes
            self.models[variable] = model
            self.anomaly_detectors[variable] = anomaly_detector
            self.scalers[variable] = scaler
            self.metrics[variable] = metrics
            self.baselines[variable] = baseline
            
            return {
                'metrics': metrics,
                'baseline': baseline,
                'n_samples': len(y_clean)
            }
            
        except Exception as e:
            self.logger.error(f"Error entrenando {variable}: {str(e)}")
            raise

    def _prepare_features(self, data: pd.DataFrame, variable: str) -> Tuple[np.ndarray, np.ndarray]:
        """Prepara features mejorados para el modelo."""
        features = []
        
        # 1. Features temporales mejorados
        hour = np.array(data.index.hour)
        dayofweek = np.array(data.index.dayofweek)
        month = np.array(data.index.month)
        is_weekend = np.array(dayofweek >= 5, dtype=int)
        
        features.extend([
            hour.reshape(-1, 1),
            dayofweek.reshape(-1, 1),
            month.reshape(-1, 1),
            is_weekend.reshape(-1, 1),
            np.sin(2 * np.pi * hour/24).reshape(-1, 1),
            np.cos(2 * np.pi * hour/24).reshape(-1, 1)
        ])
        
        # 2. Lags con validación
        for lag in self.config.lags:
            lag_values = data[variable].shift(lag).to_numpy()
            features.append(lag_values.reshape(-1, 1))
        
        # 3. Estadísticas móviles mejoradas
        rolling = data[variable].rolling(self.config.window_size)
        features.extend([
            rolling.mean().to_numpy().reshape(-1, 1),
            rolling.std().to_numpy().reshape(-1, 1),
            rolling.min().to_numpy().reshape(-1, 1),
            rolling.max().to_numpy().reshape(-1, 1),
            rolling.skew().to_numpy().reshape(-1, 1),
            rolling.kurt().to_numpy().reshape(-1, 1)
        ])
        
        # 4. Features de tendencia
        rolling_diffs = data[variable].diff().rolling(self.config.window_size)
        features.extend([
            rolling_diffs.mean().to_numpy().reshape(-1, 1),
            rolling_diffs.std().to_numpy().reshape(-1, 1)
        ])
        
        # Combinar y limpiar
        X = np.hstack(features)
        y = data[variable].to_numpy()
        
        # Manejo mejorado de valores faltantes
        mask = ~np.isnan(X).any(axis=1) & ~np.isnan(y)
        X_clean = X[mask]
        y_clean = y[mask]
        
        # Validación adicional
        if len(X_clean) < self.config.window_size:
            raise ValueError(f"Insuficientes datos para variable {variable}")
            
        return X_clean, y_clean

    def _train_anomaly_detector(self, X: np.ndarray) -> IsolationForest:
        """Entrena detector de anomalías."""
        try:
            detector = IsolationForest(
                contamination=self.config.anomaly_contamination,
                random_state=42,
                n_jobs=-1
            )
            detector.fit(X)
            return detector
        except Exception as e:
            self.logger.error(f"Error en detector de anomalías: {str(e)}")
            raise

    def _train_predictor(self, X: np.ndarray, y: np.ndarray) -> GradientBoostingRegressor:
        """Entrena modelo predictivo con configuración optimizada."""
        model = GradientBoostingRegressor(
            n_estimators=200,          # Aumentado de 100
            learning_rate=0.05,        # Reducido de 0.1
            max_depth=4,              # Aumentado de 3
            min_samples_split=5,      # Nuevo parámetro
            min_samples_leaf=3,       # Nuevo parámetro
            subsample=0.8,            # Nuevo parámetro
            random_state=42,
            validation_fraction=0.1    # Nuevo parámetro
        )
        
        try:
            model.fit(X, y)
            return model
        except Exception as e:
            self.logger.error(f"Error en entrenamiento del predictor: {str(e)}")
            raise

    def _calculate_baseline(self, series: pd.Series) -> Dict[str, float]:
        """Calcula línea base y límites."""
        stats_dict = {
            'mean': float(series.mean()),
            'std': float(series.std()),
            'q25': float(series.quantile(0.25)),
            'q75': float(series.quantile(0.75))
        }
        
        stats_dict.update({
            'lower_limit': stats_dict['q25'] - 1.5 * (stats_dict['q75'] - stats_dict['q25']),
            'upper_limit': stats_dict['q75'] + 1.5 * (stats_dict['q75'] - stats_dict['q25'])
        })
        
        return stats_dict

    def _validate_model(self, model: GradientBoostingRegressor, X: np.ndarray, y: np.ndarray) -> Dict[str, float]:
        """Valida modelo con cross-validation temporal."""
        cv = TimeSeriesSplit(n_splits=self.config.n_splits)
        metrics = {
            'mse': [],
            'mae': [],
            'precision': [],
            'recall': []
        }
        
        for train_idx, val_idx in cv.split(X):
            X_train, X_val = X[train_idx], X[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            model.fit(X_train, y_train)
            pred = model.predict(X_val)
            
            metrics['mse'].append(mean_squared_error(y_val, pred))
            metrics['mae'].append(mean_absolute_error(y_val, pred))
            
            # Métricas de clasificación
            thresh = np.percentile(y_train, 95)
            y_binary = (y_val > thresh).astype(int)
            pred_binary = (pred > thresh).astype(int)
            
            metrics['precision'].append(precision_score(y_binary, pred_binary))
            metrics['recall'].append(recall_score(y_binary, pred_binary))
        
        return {k: float(np.mean(v)) for k, v in metrics.items()}
    
    def _validate_training(self, results: Dict[str, Dict[str, Any]]) -> None:
        """
        Valida resultados del entrenamiento.
        
        Args:
            results: Resultados por variable
        """
        try:
            for var, res in results.items():
                metrics = res['metrics']
                
                # Verificar precisión
                if metrics['precision'] < self.config.target_precision:
                    self.logger.warning(
                        f"Precisión baja para {var}: {metrics['precision']:.3f} "
                        f"(objetivo: {self.config.target_precision})"
                    )
                
                # Verificar recall
                if metrics['recall'] < self.config.target_recall:
                    self.logger.warning(
                        f"Recall bajo para {var}: {metrics['recall']:.3f} "
                        f"(objetivo: {self.config.target_recall})"
                    )
                
                # Verificar MSE
                if metrics['mse'] > np.mean([r['metrics']['mse'] for r in results.values()]) * 2:
                    self.logger.warning(
                        f"MSE alto para {var}: {metrics['mse']:.4f}"
                    )
                    
                self.logger.info(f"Validación completada para {var}")
                
        except Exception as e:
            self.logger.error(f"Error en validación de entrenamiento: {str(e)}")
            raise

    def predict(self, data: pd.DataFrame) -> Dict[str, Any]:
        """Realiza predicciones para todas las variables."""
        results = {
            'predictions': {},
            'alerts': [],
            'timestamp': data.index[-1]
        }
        
        for variable in self.models:
            prediction = self._predict_variable(data, variable)
            results['predictions'][variable] = prediction
            
            if self._should_generate_alert(prediction):
                alert = self._generate_alert(variable, prediction)
                results['alerts'].append(alert)
        
        self._update_history(results)
        return results

    def _predict_variable(self, data: pd.DataFrame, variable: str) -> Dict[str, Any]:
        """Realiza predicción para una variable."""
        try:
            X, _ = self._prepare_features(data, variable)
            X_scaled = self.scalers[variable].transform(X[-1:])
            
            value = float(self.models[variable].predict(X_scaled)[0])
            intervals = self._get_prediction_intervals(self.models[variable], X_scaled)
            anomaly_score = float(self.anomaly_detectors[variable].score_samples(X[-1:])[0])
            
            return {
                'value': value,
                'intervals': intervals,
                'anomaly_score': anomaly_score,
                'timestamp': data.index[-1]
            }
        except Exception as e:
            self.logger.error(f"Error en predicción: {str(e)}")
            raise

    def _get_prediction_intervals(self, model: GradientBoostingRegressor, X: np.ndarray) -> Dict[str, float]:
        """Calcula intervalos de predicción."""
        try:
            pred = model.predict(X)
            predictions = []
            
            for estimator in model.estimators_:
                predictions.append(estimator[0].predict(X))
            
            predictions = np.array(predictions).flatten()
            
            lower = float(np.percentile(predictions, (1 - self.config.confidence_level) * 50))
            upper = float(np.percentile(predictions, (1 + self.config.confidence_level) * 50))
            
            return {'lower': lower, 'upper': upper}
            
        except Exception as e:
            self.logger.error(f"Error en cálculo de intervalos: {str(e)}")
            return {
                'lower': float(pred[0] * 0.9),
                'upper': float(pred[0] * 1.1)
            }

    def _should_generate_alert(self, prediction: Dict[str, Any]) -> bool:
        """Determina si se debe generar una alerta con criterios mejorados."""
        # 1. Score de anomalía más estricto
        anomaly_alert = prediction['anomaly_score'] < -self.config.alert_threshold

        # 2. Desviación del intervalo de predicción
        value = prediction['value']
        lower = prediction['intervals']['lower']
        upper = prediction['intervals']['upper']
        interval_deviation = (
            value < lower - abs(lower * 0.1) or  # 10% de margen
            value > upper + abs(upper * 0.1)
        )

        # 3. Cambio brusco respecto a histórico
        if prediction['variable'] in self.predictions_history:
            history = self.predictions_history[prediction['variable']]
            if history:
                last_value = history[-1]['value']
                change_ratio = abs((value - last_value) / last_value)
                sudden_change = change_ratio > 0.3  # 30% de cambio
            else:
                sudden_change = False
        else:
            sudden_change = False

        return anomaly_alert or (interval_deviation and sudden_change)

    def _generate_alert(self, variable: str, prediction: Dict[str, Any]) -> Dict[str, Any]:
        """Genera alerta basada en predicción."""
        severity = max(
            -prediction['anomaly_score'],
            abs(prediction['value'] - prediction['intervals']['upper']) / prediction['value'],
            abs(prediction['value'] - prediction['intervals']['lower']) / prediction['value']
        )
            
        return {
            'timestamp': prediction['timestamp'],
            'variable': variable,
            'severity': float(severity),
            'message': f"Anomalía detectada en {variable}",
            'details': prediction
        }

    def _update_history(self, results: Dict[str, Any]) -> None:
        """Actualiza histórico de predicciones y alertas."""
        timestamp = results['timestamp']
        
        for var, pred in results['predictions'].items():
            if var not in self.predictions_history:
                self.predictions_history[var] = []
            self.predictions_history[var].append({
                'timestamp': timestamp,
                'value': pred['value'],
                'anomaly_score': pred['anomaly_score']
            })
        
        self.alerts_history.extend(results['alerts'])
        
        # Mantener solo último día
        cutoff = timestamp - pd.Timedelta(days=1)
        for var in self.predictions_history:
            self.predictions_history[var] = [
                p for p in self.predictions_history[var]
                if p['timestamp'] > cutoff
            ]
        self.alerts_history = [
            a for a in self.alerts_history
            if a['timestamp'] > cutoff
        ]

def test_prognosis_system(data: pd.DataFrame, variables: List[str]):
    """Prueba el sistema de prognosis."""
    print("\n=== PRUEBA DE SISTEMA DE PROGNOSIS ===\n")
    
    try:
        # Inicializar y entrenar
        system = PrognosisSystem()
        training_results = system.train_models(data, variables)
        
        # Mostrar resultados de entrenamiento
        print("\nResultados de Entrenamiento:")
        for var, res in training_results.items():
            metrics = res['metrics']
            print(f"\n{var}:")
            print(f"  Precisión: {metrics['precision']:.3f}")
            print(f"  Recall: {metrics['recall']:.3f}")
            print(f"  MSE: {metrics['mse']:.4f}")
        
        # Realizar predicciones
        recent_data = data.iloc[-100:]
        results = system.predict(recent_data)
        
        # Mostrar predicciones
        print("\nPredicciones:")
        for var, pred in results['predictions'].items():
            print(f"\n{var}:")
            print(f"  Valor: {pred['value']:.3f}")
            print(f"  Intervalo: [{pred['intervals']['lower']:.3f}, {pred['intervals']['upper']:.3f}]")
            print(f"  Score Anomalía: {pred['anomaly_score']:.3f}")
        
        if results['alerts']:
            print("\nAlertas:")
            for alert in results['alerts']:
                print(f"- {alert['message']} (Severidad: {alert['severity']:.2f})")
        
        return system
        
    except Exception as e:
        print(f"Error en pruebas: {str(e)}")
        return None

# Ejecución condicional
if 'processed_data' in globals() and 'variable_analyzer' in globals():
    if processed_data is not None and variable_analyzer is not None:
        prognosis_system = test_prognosis_system(
            processed_data,
            list(variable_analyzer.results['selected_variables'].keys())
        )
    else:
        print("Error: Ejecutar primero fases anteriores")
else:
    print("Error: Ejecutar primero fases anteriores")

2024-12-27 10:00:21,237 - __main__ - INFO - Iniciando entrenamiento para 11 variables
2024-12-27 10:00:21,237 - INFO - Iniciando entrenamiento para 11 variables



=== PRUEBA DE SISTEMA DE PROGNOSIS ===



2024-12-27 10:05:21,938 - __main__ - WARNING - Precisión baja para Armónicos IL1: Armónico 3 (%IL1): 0.680 (objetivo: 0.8)
2024-12-27 10:05:21,938 - WARNING - Precisión baja para Armónicos IL1: Armónico 3 (%IL1): 0.680 (objetivo: 0.8)
2024-12-27 10:05:21,941 - __main__ - WARNING - Recall bajo para Armónicos IL1: Armónico 3 (%IL1): 0.668 (objetivo: 0.75)
2024-12-27 10:05:21,941 - WARNING - Recall bajo para Armónicos IL1: Armónico 3 (%IL1): 0.668 (objetivo: 0.75)
2024-12-27 10:05:21,943 - __main__ - INFO - Validación completada para Armónicos IL1: Armónico 3 (%IL1)
2024-12-27 10:05:21,943 - INFO - Validación completada para Armónicos IL1: Armónico 3 (%IL1)
2024-12-27 10:05:21,946 - __main__ - WARNING - Precisión baja para Armónicos IL1: Armónico 2 (%IL1): 0.000 (objetivo: 0.8)
2024-12-27 10:05:21,946 - WARNING - Precisión baja para Armónicos IL1: Armónico 2 (%IL1): 0.000 (objetivo: 0.8)
2024-12-27 10:05:21,948 - __main__ - WARNING - Recall bajo para Armónicos IL1: Armónico 2 (%IL1): 0.00


Resultados de Entrenamiento:

Armónicos IL1: Armónico 3 (%IL1):
  Precisión: 0.680
  Recall: 0.668
  MSE: 0.0494

Armónicos IL1: Armónico 2 (%IL1):
  Precisión: 0.000
  Recall: 0.000
  MSE: 0.1949

Distorsión armónica: IL1 (%I THD):
  Precisión: 0.392
  Recall: 0.387
  MSE: 0.0584

Armónicos VL3: Armónico 5 (%VL3):
  Precisión: 0.191
  Recall: 0.200
  MSE: 0.0595

Armónicos VL2: Armónico 5 (%VL2):
  Precisión: 0.197
  Recall: 0.200
  MSE: 0.0724

Armónicos IL2: Armónico 7 (%IL2):
  Precisión: 0.423
  Recall: 0.554
  MSE: 0.3713

Armónicos VL1: Armónico 5 (%VL1):
  Precisión: 0.199
  Recall: 0.197
  MSE: 0.0184

Armónicos IL1: Armónico 7 (%IL1):
  Precisión: 0.461
  Recall: 0.438
  MSE: 0.0533

Corriente: L1 (A):
  Precisión: 0.000
  Recall: 0.000
  MSE: 0.0982

Armónicos IL1: Armónico 5 (%IL1):
  Precisión: 0.391
  Recall: 0.388
  MSE: 0.1890

Tensión: L2 - L3 (V):
  Precisión: 0.199
  Recall: 0.193
  MSE: 0.0741
Error en pruebas: 'variable'
